# Inspecting and cleaning the dataset

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Options to display full dataset with pandas functions (comment if not desired)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Import the dataset to be analyzed
data = pd.read_csv('data.csv', index_col='id')
#Keep a copy of the orginal file to leave it unchanged
data_original=data.copy()

In [2]:
#Inspect data
data.head()

,Unnamed: 0,sex,velo,status,animals,note__0,warning,job_type,language,latrines,birthdate,household,motorbike,number_cow,velo_value,water_well,number_porc,group_member,house_nature,oxen_revenue,pork_revenue,school_level,family_job__0,family_sex__0,main_activity,meals_per_day,registered__0,available_land,family_link__0,number_chicken,origin_village,other_revenues,total_revenues,alphabetization,cereals_revenue,children_caring,daughter_caring,motorbike_value,property_houses,wish_motivation,beneficiary_wish,children_feeding,occupied_time__0,type_of_document,work_accident__0,food_expenditures,other_expenditures,total_expenditures,unoccupied_time__0,vegetables_revenue,children_caring_how,meals_per_day_value,other_main_activity,remittances_revenue,other_revenues_value,chronical_diseases__0,family_school_left__0,group_member_benefits,property_houses_value,your_village_expenses,agricultural_equipment,birthdate_household__0,facilitator_perception,family_school_level__0,local_product_quantity,number_small_ruminants,small_business_revenue,family_activity_type__0,small_ruminants_revenue,start_livestock_animals,contruction_expenditures,other_expenditures_value,poultry_and_eggs_revenue,school_fees_expenditures,agropastoral_expenditures,medical_care_expenditures,community_life_expenditures,agricultural_equipment_value,family_school_left_reason__0,children_birth_certificate__0,livestock_products_sale_revenue,agricolture_products_sale_revenue,note__1,note__2,note__3,note__4,family_job__1,family_job__2,family_job__3,family_job__4,family_sex__1,family_sex__2,family_sex__3,family_sex__4,registered__1,registered__2,registered__3,registered__4,family_link__1,family_link__2,family_link__3,family_link__4,occupied_time__1,occupied_time__2,occupied_time__3,occupied_time__4,work_accident__1,work_accident__2,work_accident__3,work_accident__4,unoccupied_time__1,unoccupied_time__2,unoccupied_time__3,unoccupied_time__4,chronical_diseases__1,chronical_diseases__2,chronical_diseases__3,chronical_diseases__4,family_school_left__1,family_school_left__2,family_school_left__3,family_school_left__4,birthdate_household__1,birthdate_household__2,birthdate_household__3,birthdate_household__4,family_school_level__1,family_school_level__2,family_school_level__3,family_school_level__4,family_activity_type__1,family_activity_type__2,family_activity_type__3,family_activity_type__4,family_school_left_reason__1,family_school_left_reason__2,family_school_left_reason__3,family_school_left_reason__4,children_birth_certificate__1,children_birth_certificate__2,children_birth_certificate__3,children_birth_certificate__4,note__5,family_job__5,family_sex__5,registered__5,family_link__5,occupied_time__5,work_accident__5,unoccupied_time__5,chronical_diseases__5,family_school_left__5,birthdate_household__5,family_school_level__5,family_activity_type__5,family_school_left_reason__5,children_birth_certificate__5,$value,note__6,family_job__6,family_sex__6,registered__6,family_link__6,occupied_time__6,work_accident__6,unoccupied_time__6,chronical_diseases__6,family_school_left__6,birthdate_household__6,family_school_level__6,family_activity_type__6,family_school_left_reason__6,children_birth_certificate__6,note__7,family_job__7,family_sex__7,registered__7,family_link__7,occupied_time__7,work_accident__7,unoccupied_time__7,chronical_diseases__7,family_school_left__7,birthdate_household__7,family_school_level__7,family_activity_type__7,family_school_left_reason__7,children_birth_certificate__7,note__8,family_job__8,family_sex__8,registered__8,family_link__8,occupied_time__8,work_accident__8,unoccupied_time__8,chronical_diseases__8,family_school_left__8,birthdate_household__8,family_school_level__8,family_activity_type__8,family_school_left_reason__8,children_birth_certificate__8,note__9,family_job__9,family_sex__9,registered__9,family_link__9,occupied_time__9,work_accident__9,unoccupied_time__9,chronical_diseases__9,family_school_left__9,birthdate_household__9,family_school_level__9,family_ac

In [3]:
#Check shape of dataset: row x columns
data.shape

(1737, 219)

Save multiplicity of categorical-values columns, i.e. cardinality, in a dictionary called card_dict, which can be addressed as card_dict[column_name] <br>
Example: column 'sex' has cardinality 2 because data take two values, 'f' and 'm', so card_dict['sex']=2

In [4]:
# Select only columns with categorical values, i.e. text
object_cols = [col for col in data.columns if data[col].dtype == "object"]
# Extract the multiplicity of categorical values for each column
object_nunique = list(map(lambda col: data[col].nunique(), object_cols))
# create a dictionary of {name_column:cardinality}
card_dict = dict(zip(object_cols, object_nunique))

Check which are the columns totally empty/or with many missing values

In [5]:
#count missing values per column
missing_val_count_by_column = (data.isnull().sum())
#Check totally empty columns
print(missing_val_count_by_column[missing_val_count_by_column == len(data)])
#print(missing_val_count_by_column[missing_val_count_by_column > 1700])

note__0                         1737
warning                         1737
note__1                         1737
note__2                         1737
note__3                         1737
note__4                         1737
occupied_time__4                1737
work_accident__4                1737
note__5                         1737
occupied_time__5                1737
work_accident__5                1737
note__6                         1737
occupied_time__6                1737
work_accident__6                1737
family_school_left_reason__6    1737
note__7                         1737
occupied_time__7                1737
work_accident__7                1737
family_school_left_reason__7    1737
note__8                         1737
occupied_time__8                1737
work_accident__8                1737
note__9                         1737
occupied_time__9                1737
work_accident__9                1737
family_school_left_reason__9    1737
dtype: int64


In [6]:
#Check if there are empty rows, withouth sex or birhdate (necessary features to identify a beneficiary) and eliminate them
ind_nan=[]
for index, row in data.iterrows():
    if pd.notna(row['sex'])==False or pd.notna(row['birthdate'])==False: 
        ind_nan.append(index)
#print(ind_nan)
data.drop(ind_nan, inplace=True)

In [7]:
#Check shape of data and data_orginal to confirm that empty rows have been removed
print(data.shape, data_original.shape)

(1683, 219) (1737, 219)


## Column 'birthdate' of beneficiaries transformed into 'age' (it is easier to work with ages than dates)

In [8]:
#Check age range of beneficiaries from column 'birthdate' 
#check if there are incorrect ages (negative or less than 18 or more than 85, this max age can be changed)
print(data.shape)
age = dict()
no_age = list()
for index, row in data.iterrows():
    age[index] = 0

    elem = str(row['birthdate']).split('-')    
       
    if(pd.notna(row['birthdate'])):
        year = elem[0]
        age[index] = 2019 - int(year) 
        if age[index] < 18 or age[index] > 85: 
            no_age.append(index)
            #print(index, row['birthdate'], age[index])
    if pd.notna(row['birthdate'])==False: no_age.append(index)
        
data['age'] = age.values()
print('How many ages values?',len(age.values()))
print('How many incorrect ages (negative or less than 18 or more than 85)?',len(no_age))

(1683, 219)
How many ages values? 1683
How many incorrect ages (negative or less than 18 or more than 85)? 12


In [9]:
#Drop incorrect ages and check min and max age of beneficiaries
data.drop(no_age, inplace=True)
print('Min and Max ages:', data['age'].min(), data['age'].max())

Min and Max ages: 18 82


In [10]:
#Drop column 'birthdate', transformed into 'age'
data.drop('birthdate', axis=1, inplace=True)

In [11]:
#Check shape of data and data_orginal to confirm that empty rows have been removed
print(data.shape, data_original.shape)

(1671, 219) (1737, 219)


## Column 'sex' transformed into numbers (f,m) --> (0,1)
When building a classifier it is always easier to work with numerical values instead of categorical values

In [12]:
#Print cardinality (unique entry values) for column 'sex' and print the specific values
print(card_dict['sex'],data.sex.unique())
# Count rows for each unique value
x = data[['Unnamed: 0','sex']].groupby('sex').count()
x.sort_values(by=['Unnamed: 0'])

2 ['f' 'm']


,Unnamed: 0
sex,
m,4
f,1667


In [13]:
#Transform categorical values into numbers: sex (f,m)==>(0,1)
for index in data.index: 
    data.at[index,'sex']=0 if data.at[index,'sex'] == 'f' else 1

#check that transfomation has not changed the counting by groups (nan values will sum to group 0)
x = data[['Unnamed: 0','sex']].groupby('sex').count()
x.sort_values(by=['Unnamed: 0'])

,Unnamed: 0
sex,
1,4
0,1667


## Column 'language' --> 'num_languages' 
Create new column which counts the number of alphabetization languages ==> (0,1,2,3)

In [14]:
##Check cardinality (unique entries value) for column 'language' and the specific values
print(card_dict['language'],data.language.unique())
x = data[['Unnamed: 0','language']].groupby('language').count()
x.sort_values(by=['Unnamed: 0'])

9 ['[]' "['french']" "['lyele-nuni ']" "['moore']"
 "['lyele-nuni ', 'french']" "['french', 'moore', 'lyele-nuni ']"
 "['french', 'lyele-nuni ']" "['moore', 'french']"]


,Unnamed: 0
language,
"['french', 'moore', 'lyele-nuni ']",1
"['french', 'lyele-nuni ']",3
"['lyele-nuni ', 'french']",4
"['moore', 'french']",4
['lyele-nuni '],66
['moore'],73
['french'],88
[],1432


In [15]:
#Print alphabetization languages to an output file
for index in data.index:
    if data.at[index,'language'] == "['lyele-nuni ', 'french']": 
        data.at[index,'language']="['french', 'lyele-nuni ']"

x = data[['Unnamed: 0','language']].groupby('language').count()
counts = x.sort_values(by=['Unnamed: 0'])
counts.rename(columns={'Unnamed: 0' : 'counts'}, inplace=True)
#print data to output file 'languages.csv' (store in a folder output_files)
counts.to_csv('output_files/languages.csv')

In [16]:
#Transform 'languages' into numbers of known languages (0,1,2,3)
num_languages={}
for index, row in data.iterrows():
    num_languages[index]=0
        
    elem=(row['language']).split(',')
        
    if len(elem) == 1 and elem[0] == '[]':
        num_languages[index] = 0
    elif len(elem) == 1 and elem[0] != '[]':
        num_languages[index] = 1
    elif len(elem) == 2:
        num_languages[index] = 2
    elif len(elem) == 3:
        num_languages[index] = 3
        
    #if row['language'] == '[]': print(index,len(elem),elem)
        
data['num_languages']=num_languages.values()
x = data[['Unnamed: 0','num_languages']].groupby('num_languages').count()
x.sort_values(by=['Unnamed: 0'])

,Unnamed: 0
num_languages,
3,1
2,11
1,227
0,1432


In [17]:
#Drop column 'language' transformed into new column 'num_languages'
data.drop('language', axis=1, inplace=True)
print(data.shape, data_original.shape)

(1671, 219) (1737, 219)


## Column 'school_level' of beneficiaries transformed into numbers 
 (never,pre-school,primary,secondary)-->(0,1,2,3)

In [18]:
#Check cardinality from column 'scool_level'
print(card_dict['school_level'],data.school_level.unique())
x = data[['Unnamed: 0','school_level']].groupby('school_level').count()
x.sort_values(by=['school_level'])

4 ['never' 'primary' 'secondary' 'pre_school' nan]


,Unnamed: 0
school_level,
never,1488
pre_school,2
primary,138
secondary,41


In [19]:
#Transform colum 'school_level' into numbers
level={}
for index,row in data.iterrows():
    level[index]=0
        
    if row['school_level'] == 'never' or pd.notna(row['school_level'])==False:
        level[index]=0
    elif row['school_level'] == 'pre_school':
        level[index]=1
    elif row['school_level']== 'primary':
        level[index]=2
    elif row['school_level']== 'secondary':
        level[index]=3
        
data['school_level']=level.values()
x = data[['Unnamed: 0','school_level']].groupby('school_level').count()
x.sort_values(by=['school_level'])

,Unnamed: 0
school_level,
0,1490
1,2
2,138
3,41


### Checking categorical values and cardinality in columns: 'family_link__', 'family_sex__', 'children_birth_certificate__', 'registered__', 'family_school_level__','family_school_left__',etc.
Un/comment the information that one wants to display/hide

In [20]:
#Check family link types of beneficiary from column 'family_link__'
print(card_dict[str('family_link__0')],data.family_link__0.unique())
x = data[['Unnamed: 0','family_link__0']].groupby('family_link__0').count()
x.sort_values(by=['Unnamed: 0'])

7 ['son' 'nephew' nan 'cousin' 'son in law' 'father' 'uncle' 'mother']


,Unnamed: 0
family_link__0,
father,1
mother,1
uncle,1
son in law,3
cousin,6
nephew,143
son,1492


In [21]:
#Check family sex types of beneficiary from column 'family_sex__'
print(card_dict[str('family_sex__0')],data.family_sex__5.unique())
x = data[['Unnamed: 0','family_sex__0']].groupby('family_sex__0').count()
x.sort_values(by=['Unnamed: 0'])

2 [nan 'm' 'f']


,Unnamed: 0
family_sex__0,
f,811
m,837


In [22]:
#Check 'children_birth_certificate__' values
print(card_dict[str('children_birth_certificate__0')],data.children_birth_certificate__0.unique())
x = data[['Unnamed: 0','children_birth_certificate__0']].groupby('children_birth_certificate__0').count()
x.sort_values(by=['Unnamed: 0'])

3 ['yes_delivery' 'yes_after_years' 'no' nan]


,Unnamed: 0
children_birth_certificate__0,
no,115
yes_after_years,729
yes_delivery,812


In [23]:
#Check 'registered__' values
print(card_dict[str('registered__0')],data.registered__0.unique())
x = data[['Unnamed: 0','registered__0']].groupby('registered__0').count()
x.sort_values(by=['Unnamed: 0'])

2 ['y' 'n' nan]


,Unnamed: 0
registered__0,
n,114
y,1538


In [24]:
#Check school level of family member from column 'family_scool_level__'
print(card_dict['family_school_level__0'],data.family_school_level__0.unique())
x = data[['Unnamed: 0','family_school_level__0']].groupby('family_school_level__0').count()
x.sort_values(by=['family_school_level__0'])

4 ['never' 'primary' 'secondary' nan 'pre_school']


,Unnamed: 0
family_school_level__0,
never,212
pre_school,8
primary,945
secondary,459


In [25]:
#Check if family member left school from column 'family_scool_left__'
print(card_dict['family_school_left__0'],data.family_school_left__0.unique())
x = data[['Unnamed: 0','family_school_left__0']].groupby('family_school_left__0').count()
x.sort_values(by=['Unnamed: 0'])

2 ['n' 'y' nan]


,Unnamed: 0
family_school_left__0,
y,98
n,1477


In [26]:
#Check if family member has a job from column 'family_job__'
print(card_dict['family_job__0'],data.family_job__0.unique())
x = data[['Unnamed: 0','family_job__0']].groupby('family_job__0').count()
x.sort_values(by=['Unnamed: 0'])

2 ['n' nan 'y']


,Unnamed: 0
family_job__0,
y,17
n,1623


In [27]:
#Check if family member has a job from column 'family_job__'
#print(card_dict['chronical_diseases__0'],data.chronical_diseases__0.unique())
#x = data[['Unnamed: 0','chronical_diseases__0']].groupby('chronical_diseases__0').count()
#x.sort_values(by=['chronical_diseases__0'])

In [28]:
#Check activity type of family member from column 'family_activity_type__'
#print(card_dict['family_activity_type__0'],data.family_activity_type__0.unique())
#x = data[['Unnamed: 0','family_activity_type__0']].groupby('family_activity_type__0').count()
#x.sort_values(by=['family_activity_type__0'])

In [29]:
#Check reason family member left school from column 'family_scool_left_reason__'
#print(card_dict['family_school_left_reason__0'],data.family_school_left_reason__0.unique())
#x = data[['Unnamed: 0','family_school_left_reason__0']].groupby('family_school_left_reason__0').count()
#x.sort_values(by=['family_school_left_reason__0'])

In [30]:
#Check family member occupied and uncoppied time 'occupied_time__' and 'unoccupied_time__'
#x = data[['Unnamed: 0','occupied_time__0']].groupby('occupied_time__0').count()
#x.sort_values(by=['occupied_time__0'])

## Analyzing data about kids: number, registered, certified, school level, left school, etc.
1. This part of the code reorganizes the data corresponding to kids in more concise and clear columns
2. Separate files containing text from specific columns are created, ex. 'kids_activity_type.txt'; these files can be further analyzed with language processing libraries
2. A separate dataset with data corresponding only to kids is also created 'data_kids.csv'

In [31]:
#Count number of kids from columns "birthdate_household__", check that age < 18 to count to number of kids
#Count number of boys, girls or not assigned sex
#Count number of registered kids
#Count birth certificate of kids
#Count kids school level
#Count kids who left school and why
#Count number of adults and those who left school
 
#For filling a dataset only for kids
#Declare the columns to have in the new dataset
col_kids=['age','sex','registered','birth_certified','school_level','school_left','school_left_reason'
         ,'job','job_type','occupied_time','unoccupied_time','work_accident','chronic_disease']
list_data_kids=[] 

#Initializing empty dictionaries and lists

#Total kids
num_kids={}
num_kids_reg={}
num_kids_cert={}
num_kids_noschool={}
num_kids_newborn={}
num_kids_preschool={}
num_kids_primary={}
num_kids_secondary={}
num_kids_left_school={}
#num_kids_work_accident={}
#num_kids_chronic_disease={}

#Girls
num_girls={}
num_girls_reg={}
num_girls_cert={}
num_girls_noschool={}
num_girls_newborn={}
num_girls_preschool={}
num_girls_primary={}
num_girls_secondary={}
num_girls_left_school={}
#num_girls_work_accident={}
#num_girls_chronic_disease={}

#Boys
num_boys={}

#Kids not assigned sex
num_kids_na={} 

#Adults
num_adults={}
num_adults_left_school={}

#Reasons for leaving school
kids_reason_left_school=[]
girls_reason_left_school=[]
boys_reason_left_school=[]

#Activity type of kids with job
kids_activity_type=[]
girls_activity_type=[]
#Activity type of kids who left school
kids_left_school_activity_type=[]
girls_left_school_activity_type=[]

#Counting total number of kids (girls) with schooling age
k_num_schooling_age=0
g_num_schooling_age=0

for index, row in data.iterrows():
    k_num=0
    k_reg=0
    k_cert=0
    k_noschool=0
    k_newborn=0
    k_preschool=0
    k_primary=0
    k_secondary=0
    k_left_school=0
    
#    k_work_accident=0
#    k_disease=0
    
    g_num=0
    g_reg=0
    g_cert=0
    g_noschool=0
    g_newborn=0
    g_preschool=0
    g_primary=0
    g_secondary=0
    g_left_school=0
    
#    g_work_accident=0
#    g_disease=0
    
    b_num=0
      
    kna_num=0
    
    n_adults=0
    n_adults_left_school=0
     
    i=0
    
    num_kids[index]=0
    num_kids_reg[index]=0
    num_kids_cert[index]=0
    num_kids_noschool[index]=0
    num_kids_newborn[index]=0
    num_kids_preschool[index]=0
    num_kids_primary[index]=0
    num_kids_secondary[index]=0
    num_kids_left_school[index]=0
#   num_kids_work_accident[index]=0
#   num_kids_chronic_disease[index]=0
        
    num_girls[index] = 0
    num_girls_reg[index] = 0
    num_girls_cert[index] = 0
    num_girls_noschool[index]=0
    num_girls_newborn[index]=0
    num_girls_preschool[index]=0
    num_girls_primary[index]=0
    num_girls_secondary[index]=0
    num_girls_left_school[index]=0
#   num_girls_work_accident[index]=0
#   num_girls_chronic_disease[index]=0
        
    num_boys[index] = 0
        
    num_kids_na[index] = 0
        
    num_adults[index]=0
    num_adults_left_school[index]=0
    
    # each household can have up to 10 family members
    for i in range(10):
        i = str(i) 
        #data['family_school_level__'+i]=data['family_school_level__'+i].replace(np.nan,'never')
        
        if pd.notna(row['birthdate_household__'+i]):
            
            #calculate age
            elem = str(row['birthdate_household__'+i]).split('-')
            age = 2019 - int(elem[0])
            
            #Check age to count to kids (below/equal to 16 years)
            if age <= 16: 
                
                #Create dataset only for kids
                list_data_kids.append([age,row['family_sex__'+i],row['registered__'+i]
                                  ,row['children_birth_certificate__'+i]
                                  ,row['family_school_level__'+i]
                                  ,row['family_school_left__'+i]
                                  ,row['family_school_left_reason__'+i]
                                  ,row['family_job__'+i],row['family_activity_type__'+i]
                                  ,row['occupied_time__'+i],row['unoccupied_time__'+i]
                                  ,row['work_accident__'+i],row['chronical_diseases__'+i]])
                
                k_num +=1
                
                #Count registered kids
                if row['registered__'+i] == 'y': 
                    k_reg += 1
                                
                #Count birth certified kids
                if (row['children_birth_certificate__'+i] == 'yes_delivery' \
                or row['children_birth_certificate__'+i] =='yes_after_years'): 
                    k_cert +=1
                                       
                #check age to count newborn (below of 6 years)    
                if age < 6 : 
                    k_newborn += 1
                #check age to count schooling age kids (above/equal to 6 years)
                elif age >= 6: 
                    k_num_schooling_age +=1
                        
                    #counting kids who have never been to school
                    if (row['family_school_level__'+i] =='never' 
                        or pd.isna(row['family_school_level__'+i])):k_noschool +=1   
                        
                    #counting kids in school and who did not leave
                    if row['family_school_left__'+i] == 'n' or pd.isna(row['family_school_left__'+i]):
                    #check schooling level
                        if row['family_school_level__'+i]=='pre-school':
                            k_preschool +=1
                        elif row['family_school_level__'+i]=='primary':
                            k_primary +=1
                        elif row['family_school_level__'+i]=='secondary':
                            k_secondary +=1
                        
                    #count kids who left school
                    elif row['family_school_left__'+i] == 'y':
                        k_left_school +=1
                        kids_reason_left_school.append(str(row['family_school_left_reason__'+i]))
                        #kids_left_school_activity_type.append(str(row['family_activity_type__'+i]))
                        #if row['work_accident__'+i] == 'y': k_work_accident += 1
                
                    #if row['family_school_level__'+i]=='never' and row['family_school_left__'+i] == 'y': print(index)
                    
                    kids_activity_type.append(str(row['family_activity_type__'+i]))                    
                        
                #if row['chronical_diseases__'+i] == 'y': k_disease +=1
            
                #Count only for girls
                if row['family_sex__'+i] == 'f': 
                    g_num += 1 
                             
                    #Count registered girls
                    if row['registered__'+i] == 'y': 
                        g_reg += 1
                        
                    #Count birth certified girls    
                    if row['children_birth_certificate__'+i] == ('yes_delivery' or 'yes_after_years'): 
                        g_cert +=1
                        
                    #check age to count newborn girls (below of 6 years)    
                    if age < 6 : 
                        g_newborn += 1
                    #check age to count schooling age girls (above/equal to 6 years)
                    elif age >= 6: 
                        g_num_schooling_age +=1
                        
                        #count girls who have never been to school
                        if (row['family_school_level__'+i]=='never' 
                            or pd.isna(row['family_school_level__'+i])):g_noschool +=1
                                
                        #counting girls in school and who did not leave
                        if row['family_school_left__'+i] == 'n' or pd.isna(row['family_school_left__'+i]):
                            if row['family_school_level__'+i]=='pre-school':
                                g_preschool +=1
                            elif row['family_school_level__'+i]=='primary':
                                g_primary +=1
                            elif row['family_school_level__'+i]=='secondary':
                                g_secondary +=1
                                
                        #count girls who left school
                        elif row['family_school_left__'+i] == 'y':
                            g_left_school +=1
                            girls_reason_left_school.append(str(row['family_school_left_reason__'+i]))
                            #girls_left_school_activity_type.append(str(row['family_activity_type__'+i]))
                            #if row['work_accident__'+i] == 'y': g_work_accident += 1
                        
                        
                        girls_activity_type.append(str(row['family_activity_type__'+i])) 
                                        
                elif row['family_sex__'+i] == 'm': 
                    b_num += 1
                    
                elif pd.isna(row['family_sex__'+i]):
                    kna_num += 1
                    
            #if n_kids != n_boys+n_girls+n_not: print(index,n_kids,n_boys,n_girls)
            
            else:
                n_adults +=1
                if row['family_school_left__'+i] == 'y':
                    n_adults_left_school +=1

    
    num_kids[index] = k_num
    num_kids_reg[index] = k_reg
    num_kids_cert[index] = k_cert
    num_kids_noschool[index] = k_noschool
    num_kids_newborn[index] = k_newborn
    num_kids_preschool[index] = k_preschool
    num_kids_primary[index] = k_primary
    num_kids_secondary[index] = k_secondary
    num_kids_left_school[index] = k_left_school
#    num_kids_work_accident[index]= k_work_accident
#    num_kids_chronic_disease[index]= k_disease
#    num_kids_double[index]=k_double_counted
    
    num_girls[index] = g_num
    num_girls_reg[index] = g_reg
    num_girls_cert[index] = g_cert
    num_girls_noschool[index] = g_noschool
    num_girls_newborn[index] = g_newborn
    num_girls_preschool[index] = g_preschool
    num_girls_primary[index] = g_primary
    num_girls_secondary[index] = g_secondary
    num_girls_left_school[index] = g_left_school
#    num_girls_work_accident[index]= g_work_accident
#    num_girls_chronic_disease[index]= g_disease
    
    
    num_boys[index] = b_num
    num_kids_na[index] = kna_num
    num_adults[index]=n_adults
    num_adults_left_school[index]=n_adults_left_school
    
    
data['num_kids']=num_kids.values()
data['num_kids_registered']=num_kids_reg.values()
data['num_kids_certified']=num_kids_cert.values()
data['num_kids_noschool']=num_kids_noschool.values()
data['num_kids_newborn']=num_kids_newborn.values()
data['num_kids_preschool']=num_kids_preschool.values()
data['num_kids_primary']=num_kids_primary.values()
data['num_kids_secondary']=num_kids_secondary.values()
data['num_kids_left_school']=num_kids_left_school.values()
#data['num_kids_work_accident']=num_kids_work_accident.values()
#data['num_kids_chronic_disease']=num_kids_chronic_disease.values()

data['num_girls']=num_girls.values()
data['num_girls_registered']=num_girls_reg.values()
data['num_girls_certified']=num_girls_cert.values()
data['num_girls_noschool']=num_girls_noschool.values()
data['num_girls_newborn']=num_girls_newborn.values()
data['num_girls_preschool']=num_girls_preschool.values()
data['num_girls_primary']=num_girls_primary.values()
data['num_girls_secondary']=num_girls_secondary.values()
data['num_girls_left_school']=num_girls_left_school.values()
#data['num_girls_work_accident']=num_girls_work_accident.values()
#data['num_girls_chronic_disease']=num_girls_chronic_disease.values()

data['num_boys']=num_boys.values()
data['num_kids_na']=num_kids_na.values()
data['num_adults']=num_adults.values()
data['num_adults_left_school']=num_adults_left_school.values()

print('Grand total number of household members (as declared by beneficiaries):',data['household'].sum())
print('')
print('Total number of kids (age < 16):',data['num_kids'].sum())
#print('Total number of kids registered:',data['num_kids_registered'].sum())
print('Percentage of kids registered: %4.2f%%' 
      %(data['num_kids_registered'].sum()/data['num_kids'].sum()*100) )
#print('Total number of kids with birth certificate:',data['num_kids_certified'].sum())
print('Percentage of kids with birth certificate: %4.2f%%' 
      %(data['num_kids_certified'].sum()/data['num_kids'].sum()*100) )
print('Total number of kids with schooling age:',k_num_schooling_age)
#print('Total number of newborn:',data['num_kids_newborn'].sum())
print('Percentage of kids with no school level (age between 6 and 16): %4.2f%%'
      %(data['num_kids_noschool'].sum()/k_num_schooling_age*100) )
print('Percentage of kids with preschool level: %4.2f%%' 
      %(data['num_kids_preschool'].sum()/k_num_schooling_age*100) )
print('Percentage of kids with primary school level: %4.2f%%' 
      %(data['num_kids_primary'].sum()/k_num_schooling_age*100) )
print('Percentage of kids with secondary school level: %4.2f%%'
      %(data['num_kids_secondary'].sum()/k_num_schooling_age*100) )
#print('Total number of kids who left school:',data['num_kids_left_school'].sum())
print('Percentage of kids who left school: %4.2f%%' 
      %(data['num_kids_left_school'].sum()/k_num_schooling_age*100) )
print('Percentage of kids who are out of school (never went or left): %4.2f%%' 
      %((data['num_kids_left_school'].sum()+data['num_kids_noschool'].sum())
        /k_num_schooling_age*100) )
print('')
print('Total number of girls (age < 16):',data['num_girls'].sum())
#print('Total number of girls registered:',data['num_girls_registered'].sum())
print('Percentage of girls registered: %4.2f%%'
      %(data['num_girls_registered'].sum()/data['num_girls'].sum()*100) ) 
print('Percentage of girls with birth certificate: %4.2f%%' 
      %(data['num_girls_certified'].sum()/data['num_girls'].sum()*100) )
print('Total number of girls with schooling age:',g_num_schooling_age)
print('Percentage of gilrs with no school level (age between 6 and 16): %4.2f%%'
      %(data['num_girls_noschool'].sum()/g_num_schooling_age*100) )
print('Percentage of girls with preschool level: %4.2f%%' 
      %(data['num_girls_preschool'].sum()/g_num_schooling_age*100) )
print('Percentage of girls with primary school level: %4.2f%%'
      %(data['num_girls_primary'].sum()/g_num_schooling_age*100) )
print('Percentage of girls with secondary school level: %4.2f%%'
      %(data['num_girls_secondary'].sum()/g_num_schooling_age*100) )
print('Percentage of girls who left school: %4.2f%%'
      %(data['num_girls_left_school'].sum()/g_num_schooling_age*100) )
print('Percentage of girls who are out of school (never went or left): %4.2f%%' 
      %((data['num_girls_left_school'].sum()+data['num_girls_noschool'].sum())
        /g_num_schooling_age*100) )
print('')
print('Total number of boys:',data['num_boys'].sum())
print('Total number of kids not assigned sex:',data['num_kids_na'].sum())
print('Total number of adults:',data['num_adults'].sum())
print('Total number of adults who left school:',data['num_adults_left_school'].sum())

print('Average number of kids per family:%3.0f' %(data['num_kids'].mean()) )

#printout reasons of school leaving
with open("output_files/reasons_kids_leaving_school.txt", "w") as outfile:
    outfile.write("\n".join(kids_reason_left_school))
with open("output_files/reasons_girls_leaving_school.txt", "w") as outfile:
    outfile.write("\n".join(girls_reason_left_school))
    
#printout activity type of kids leaving school
#with open("output_files/kids_left_school_activity_type.txt", "w") as outfile:
#    outfile.write("\n".join(kids_left_school_activity_type))
#with open("output_files/girls_left_school_activity_type.txt", "w") as outfile:
#    outfile.write("\n".join(girls_left_school_activity_type))
    
#printout activity type of all kids with job
with open("output_files/kids_activity_type.txt", "w") as outfile:
    outfile.write("\n".join(kids_activity_type))
with open("output_files/girls_activity_type.txt", "w") as outfile:
    outfile.write("\n".join(girls_activity_type))
    
data_kids=pd.DataFrame(list_data_kids,columns=col_kids)
data_kids.to_csv('data_kids.csv',index=False)

Grand total number of household members (as declared by beneficiaries): 5361.0

Total number of kids (age < 16): 4671
Percentage of kids registered: 86.11%
Percentage of kids with birth certificate: 86.17%
Total number of kids with schooling age: 3597
Percentage of kids with no school level (age between 6 and 16): 12.96%
Percentage of kids with preschool level: 0.00%
Percentage of kids with primary school level: 72.28%
Percentage of kids with secondary school level: 12.29%
Percentage of kids who left school: 2.31%
Percentage of kids who are out of school (never went or left): 15.26%

Total number of girls (age < 16): 2314
Percentage of girls registered: 86.65%
Percentage of girls with birth certificate: 51.17%
Total number of girls with schooling age: 1810
Percentage of gilrs with no school level (age between 6 and 16): 12.27%
Percentage of girls with preschool level: 0.00%
Percentage of girls with primary school level: 74.14%
Percentage of girls with secondary school level: 11.49%
Per

In [32]:
#check school_dropout column already present in file
data['school_dropout'].isnull().any().sum()
x = data[['Unnamed: 0','school_dropout']].groupby('school_dropout').count()
x.sort_values(by=['Unnamed: 0'])

,Unnamed: 0
school_dropout,
1,390
0,1281


In [33]:
#Checking that school_dropout column verifies the data given by beneficiary 
i=0
j=0
dropout={}
for index, row in data.iterrows():
    dropout[index]=0
    #if index==1463: print(row)
    if ( (row['num_kids_left_school'] != 0 or row['num_kids_noschool'] != 0) 
        and row['school_dropout'] == 0 ) : 
        #print(index)
        i +=1
    if row['num_kids_left_school'] != 0 or row['num_kids_noschool'] != 0: dropout[index]=1
        
    if ( (row['num_kids_left_school'] == 0  and row['num_kids_noschool'] == 0) 
        and row['school_dropout'] != 0 ) : 
        j +=1
    if row['num_kids_left_school'] == 0 and row['num_kids_noschool'] == 0: dropout[index]=0
        #print(index)
print('School dropout is incorrectly selected as 0 in', i, 'cases')
print('School dropout is incorrectly selected as 1 in', j, 'cases')

#Created new column for out-of-school kids selection
data['out_of_school']=dropout.values()
print("New column 'out_of_school' has been derived based on data given by beneficiaries")

#Checking that household number sums to the toal number of family members given by the beneficiary
k=0
for index, row in data.iterrows():
    #if index==238: print(row)
    if (row['num_kids'] + row['num_adults']) != row['household']: k +=1
print('Household number differs from beneficiary family members in', k, 'cases')
x = data[['Unnamed: 0','out_of_school']].groupby('out_of_school').count()
x.sort_values(by=['Unnamed: 0'])

School dropout is incorrectly selected as 0 in 43 cases
School dropout is incorrectly selected as 1 in 0 cases
New column 'out_of_school' has been derived based on data given by beneficiaries
Household number differs from beneficiary family members in 48 cases


,Unnamed: 0
out_of_school,
1,433
0,1238


In [34]:
#drop columns which gave totally zero values, pre-school kids
data.drop('num_kids_preschool', axis=1, inplace=True)
data.drop('num_girls_preschool', axis=1, inplace=True)

#disease=(row['chronical_diseases__'+i]).lower().split() 
#                    for elem in len(disease):
#                        split(elem)
#                        for char in elem:
#                            if                     
#                              g_disease +=1

## Here we drop columns already analyzed and transformed into other values, and incomplete/unnecessary columns

In [35]:
#Drop columns not necessary anymore:
for i in range(10):
    col_link = 'family_link__'+str(i)
    data.drop(col_link, axis=1, inplace=True)
    col_sex = 'family_sex__'+str(i)
    data.drop(col_sex, axis=1, inplace=True)
    col_cert = 'children_birth_certificate__'+str(i)
    data.drop(col_cert, axis=1, inplace=True)
    col_reg = 'registered__'+str(i)
    data.drop(col_reg, axis=1, inplace=True)
    col_school = 'family_school_level__'+str(i)
    data.drop(col_school, axis=1, inplace=True)
    col_birth = 'birthdate_household__'+str(i)
    data.drop(col_birth, axis=1, inplace=True)
    col_left = 'family_school_left__'+str(i)
    data.drop(col_left, axis=1, inplace=True)
    col_reason = 'family_school_left_reason__'+str(i)
    data.drop(col_reason, axis=1, inplace=True)  
    col_work = 'work_accident__'+str(i)
    data.drop(col_work, axis=1, inplace=True) 
    col_note = 'note__'+str(i)
    data.drop(col_note, axis=1, inplace=True)
    col_occ = 'occupied_time__'+str(i)
    data.drop(col_occ, axis=1, inplace=True)
    col_unocc = 'unoccupied_time__'+str(i)
    data.drop(col_unocc, axis=1, inplace=True)
    col_job = 'family_job__'+str(i)
    data.drop(col_job, axis=1, inplace=True)
    col_activity = 'family_activity_type__'+str(i)
    data.drop(col_activity, axis=1, inplace=True)
    col_name = 'chronical_diseases__'+str(i)
    data.drop(col_name, axis=1, inplace=True)

In [36]:
print(data.shape, data_original.shape)

(1671, 90) (1737, 219)


In [37]:
#Eliminate columns totally empty
cols_with_missing = [col for col in data.columns
                     if data[col].isnull().any()]
for col in cols_with_missing:
    if data[col].isnull().sum() == len(data):
        print(col)
        data.drop(col, axis=1, inplace=True)

warning


In [38]:
print(data.shape, data_original.shape)

(1671, 89) (1737, 219)


In [39]:
#check status of the file
data.head()

,Unnamed: 0,sex,velo,status,animals,job_type,latrines,household,motorbike,number_cow,velo_value,water_well,number_porc,group_member,house_nature,oxen_revenue,pork_revenue,school_level,main_activity,meals_per_day,available_land,number_chicken,origin_village,other_revenues,total_revenues,alphabetization,cereals_revenue,children_caring,daughter_caring,motorbike_value,property_houses,wish_motivation,beneficiary_wish,children_feeding,type_of_document,food_expenditures,other_expenditures,total_expenditures,vegetables_revenue,children_caring_how,meals_per_day_value,other_main_activity,remittances_revenue,other_revenues_value,group_member_benefits,property_houses_value,your_village_expenses,agricultural_equipment,facilitator_perception,local_product_quantity,number_small_ruminants,small_business_revenue,small_ruminants_revenue,start_livestock_animals,contruction_expenditures,other_expenditures_value,poultry_and_eggs_revenue,school_fees_expenditures,agropastoral_expenditures,medical_care_expenditures,community_life_expenditures,agricultural_equipment_value,livestock_products_sale_revenue,agricolture_products_sale_revenue,$value,school_dropout,age,num_languages,num_kids,num_kids_registered,num_kids_certified,num_kids_noschool,num_kids_newborn,num_kids_primary,num_kids_secondary,num_kids_left_school,num_girls,num_girls_registered,num_girls_certified,num_girls_noschool,num_girls_newborn,num_girls_primary,num_girls_secondary,num_girls_left_school,num_boys,num_kids_na,num_adults,num_adults_left_school,out_of_school
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
185,53,0,1.0,married,"['small_ruminants', 'chicken', 'porc']",informal,n,5.0,NaN,NaN,15000.0,n,3.0,y,banco,NaN,50000.0,0,agriculture,2.0,50.0,6.0,y,vente de bière locale,270000,n,30000.0,y,y,NaN,1.0,Prendre en charge les frais scolaires sanitaires vestimentaires de mes enfants,family_take_care,y,CNI B,15000.0,habillement,105000.0,25000.0,['csps'],500.0,élevage,100000.0,30000.0,NaN,150000.0,80.0,n,50.0,20.0,4.0,NaN,15000.0,"['small_ruminants', 'chicken', 'porc']",10000.0,25000.0,20000.0,NaN,25000.0,15000.0,15000.0,NaN,85000.0,55000.0,NaN,0,35,0,4,3,4,0,1,3,0,0,0,0,0,0,0,0,0,0,4,0,1,0,0
186,56,0,1.0,married,"['small_ruminants', 'chicken']",informal,n,2.0,NaN,NaN,25000.0,n,NaN,y,banco,NaN,NaN,0,agriculture,3.0,50.0,30.0,n,néant,220000,n,10000.0,y,y,NaN,1.0,Prendre en charge les frais scolaires sanitaires et vestimentaires de ma famille,family_take_care,y,CNI B,10000.0,habillement,50000.0,NaN,['csps'],500.0,agriculture,NaN,NaN,NaN,100000.0,80.0,n,50.0,20.0,2.0,NaN,10000.0,"['small_ruminants', 'chicken']",NaN,10000.0,200000.0,5000.0,20000.0,NaN,5000.0,NaN,210000.0,10000.0,NaN,0,31,0,2,1,1,0,1,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0
187,93,0,1.0,married,"['chicken', 'small_ruminants']",informal,n,4.0,NaN,NaN,15000.0,n,NaN,y,banco,NaN,NaN,0,agriculture,3.0,25.0,10.0,n,néant,130000,n,25000.0,y,y,NaN,1.0,Prendre en charge les frais scolaires vestimentaires et sanitaires de ma famille,family_take_care,y,CNI B,25000.0,habillement,112000.0,50000.0,['csps'],1000.0,elevage,25000.0,NaN,NaN,100000.0,80.0,n,50.0,20.0,7.0,NaN,30000.0,"['small_ruminants', 'chicken']",2000.0,20000.0,NaN,5000.0,30000.0,10000.0,20000.0,NaN,30000.0,75000.0,NaN,0,39,0,4,4,4,0,1,3,0,0,3,3,3,0,0,3,0,0,1,0,0,0,0
188,242,0,1.0,widow,"['small_ruminants', 'chicken']",informal,y,2.0,NaN,NaN,10000.0,y,NaN,y,banco,NaN,NaN,0,agriculture,3.0,25.0,8.0,n,vente de condiments,135500,n,28000.0,y,y,NaN,1.0,prendre en charge les frais alimentaires sanitaires vestimentaires et scolaires de mes enfants,family_take_care,y,CNI B,50000.0,habillement,179100.0,21500.0,['csps'],1000.0,élevage,NaN,60000.0,NaN,150000.0,80.0,n,50.0,20.0,4.0,NaN,16000.0,"['small_ruminants', 'chicken']",6000.0,22500.0,10000.0,10500.0,50000.0,5100.0,35000.0,NaN,26000.0,49500.0,NaN,0,43,0,2,2,2,0,0,2,0,0,1,1,1,0,0,1,0,0,1,0,0,0,0
189,131,0,1.0,married,['porc'],informal,y,4.0,NaN,NaN,10000.0,n,4.0,y,parpin,NaN,30000.0,0,agriculture,2.0,2

## Analyzing beneficiaries life sustainability: house, animals, revenues, economic activities, etc.

In [40]:
#Check house type of beneficiary from column 'house_nature' (this could be saved in an external file for further analysis)
data['house_nature']=data['house_nature'].str.strip()
data['house_nature']=data['house_nature'].str.lower()
print(card_dict['house_nature'],data.house_nature.unique())
x = data[['Unnamed: 0','house_nature']].groupby('house_nature').count()
#x.sort_values(by=['house_nature'])

72 ['banco' 'parpin' nan 'parpaing' 'case' 'bancp' 'tole en banco'
 'en tôle en banco' 'ban  en tôle' 'ciment' 'en dur' 'parpaings'
 'en banco' 'banca' 'parping' 'paille' 'toit' 'en toit' 'banco ķ' 'dur'
 'bonco' 'terasse' 'en paille' 'b4' 'bancok' 'en parpaings' 'banc'
 'en dur et banco' 'banco (case)' 'bad' 'banco ( case)' 'ba4' 'bancc' 'b3'
 'banko' '10' 'maison an banco' 'maison a banco' 'maison an bancaux'
 'en dure']


In [41]:
#Check houses with latrines from column 'latrines'
print(card_dict['latrines'],data.latrines.unique())
x = data[['Unnamed: 0','latrines']].groupby('latrines').count()
x.sort_values(by=['Unnamed: 0'])

2 ['n' 'y' nan]


,Unnamed: 0
latrines,
n,787
y,878


In [42]:
#Transform categorical values into numbers: latrines (n,y)==>(0,1)

for index in data.index: 
    data.at[index,'latrines']=1 if data.at[index,'latrines'] == 'y' else 0

print('Percentage of houses with latrines: %2.1f%%' %(data['latrines'].sum()/len(data)*100) )

Percentage of houses with latrines: 52.5%


In [43]:
#Check beneficiaries with animals from column 'animals'
data['animals']=data['animals'].replace(np.nan,'[]')
print(card_dict['animals'],data.animals.unique())
x = data[['Unnamed: 0','animals']].groupby('animals').count()
x.sort_values(by=['Unnamed: 0'])

22 ["['small_ruminants', 'chicken', 'porc']" "['small_ruminants', 'chicken']"
 "['chicken', 'small_ruminants']" "['porc']" "['small_ruminants']" '[]'
 "['porc', 'chicken']" "['chicken']"
 "['chicken', 'small_ruminants', 'cow']" "['chicken', 'porc']"
 "['small_ruminants', 'porc']" "['porc', 'small_ruminants']"
 "['chicken', 'porc', 'small_ruminants']" "['cow', 'small_ruminants']"
 "['cow']" "['chicken', 'cow']" "['porc', 'chicken', 'small_ruminants']"
 "['porc', 'small_ruminants', 'chicken']"
 "['small_ruminants', 'cow', 'chicken']"
 "['chicken', 'small_ruminants', 'porc']" "['cow', 'chicken']"
 "['small_ruminants', 'porc', 'chicken']"]


,Unnamed: 0
animals,
"['small_ruminants', 'cow', 'chicken']",1
"['cow', 'chicken']",1
"['cow', 'small_ruminants']",1
"['chicken', 'cow']",2
"['small_ruminants', 'porc', 'chicken']",2
"['chicken', 'small_ruminants', 'porc']",3
"['chicken', 'small_ruminants', 'cow']",4
['cow'],4
"['porc', 'small_ruminants', 'chicken']",4


In [44]:
#Check percentages for each animal stock
i=0
for index, row in data.iterrows():
    if row['animals'] == '[]': i +=1        
print('Percentage of beneficiaries with animals: %2.1f%%' %((len(data)-i)/len(data)*100))

data['number_cow']=data['number_cow'].replace(np.nan, 0)
data['number_small_ruminants']=data['number_small_ruminants'].replace(np.nan, 0)
data['number_chicken']=data['number_chicken'].replace(np.nan, 0)
data['number_porc']=data['number_porc'].replace(np.nan, 0)

data['animals_tot']= (data['number_cow']+data['number_small_ruminants']
                +data['number_chicken']+data['number_porc'])

print('Percentage of cows: %2.1f%%' %(data['number_cow'].sum()/data['animals_tot'].sum()*100 ) )
print('Percentage of small_ruminants: %2.1f%%' %(data['number_small_ruminants'].sum()/data['animals_tot'].sum()*100) )
print('Percentage of chicken: %2.1f%%' %(data['number_chicken'].sum()/data['animals_tot'].sum()*100) )
print('Percentage of porc: %2.1f%%' %(data['number_porc'].sum()/data['animals_tot'].sum()*100) )

Percentage of beneficiaries with animals: 61.1%
Percentage of cows: 0.2%
Percentage of small_ruminants: 17.2%
Percentage of chicken: 65.0%
Percentage of porc: 17.6%


In [45]:
data['main_activity']=data['main_activity'].replace(np.nan, 'no_activity')
print(card_dict['main_activity'],data.main_activity.unique())
print(data['main_activity'].value_counts()/data['main_activity'].count()*100)

5 ['agriculture' 'breeding' 'trade' 'no_activity' 'craft' 'other']
agriculture    98.503890
breeding        0.658288
trade           0.538600
no_activity     0.179533
other           0.059844
craft           0.059844
Name: main_activity, dtype: float64


In [46]:
#print(card_dict['other_main_activity'],data.other_main_activity.unique())
data['other_main_activity']=data['other_main_activity'].replace(np.nan, 'no_activity').str.lower().str.strip()
x = data[['Unnamed: 0','other_main_activity']].groupby('other_main_activity').count()
counts=x.sort_values(by=['other_main_activity'])

counts.rename(columns={'Unnamed: 0' : 'counts'}, inplace=True)
#print data to output file, long file
counts.to_csv('output_files/other_activities.csv')

In [47]:
#Check if beneficiary is memeber of group
data['group_member']=data['group_member'].replace(np.nan,'n')
print(card_dict['group_member'],data.group_member.unique())
x=data[['Unnamed: 0','group_member']].groupby('group_member').count()
x.sort_values(by=['Unnamed: 0'])

2 ['y' 'n']


,Unnamed: 0
group_member,
n,752
y,919


In [48]:
#Transform member group into numbers: (n,y)-->(0,1)

for index in data.index: 
    data.at[index,'group_member']=1 if data.at[index,'group_member'] == 'y' else 0

print('Percentage of beneficiaries who are part of a group: %2.1f' 
      %((len(data)-data['group_member'].sum())/len(data)*100) )  

Percentage of beneficiaries who are part of a group: 45.0


In [49]:
#print(card_dict['group_member_benefits'],data.group_member_benefits.unique())
data['group_member_benefits']=data['group_member_benefits'].str.lower().replace(np.nan,'none')
x = data[['Unnamed: 0','group_member_benefits']].groupby('group_member_benefits').count()
counts=x.sort_values(by=['group_member_benefits'])

counts.rename(columns={'Unnamed: 0' : 'counts'}, inplace=True)
#print data to output file, long file
counts.to_csv('output_files/group_member_benefits.csv')

In [50]:
#Check beneficiary wish
data['wish_motivation']=data['wish_motivation'].replace(np.nan,'none')
data['beneficiary_wish']=data['beneficiary_wish'].replace(np.nan,'none')
#print(card_dict['wish_motivation'],data.wish_motivation.unique())

print(card_dict['beneficiary_wish'],data.beneficiary_wish.unique())
print(data['beneficiary_wish'].value_counts()/data['beneficiary_wish'].count()*100)

5 ['family_take_care' 'other' 'deal_difficulty' 'improvements_revenues'
 'business_improvement' 'none']
family_take_care         70.795931
deal_difficulty          17.115500
business_improvement      6.642729
other                     2.094554
improvements_revenues     1.915021
none                      1.436266
Name: beneficiary_wish, dtype: float64


In [51]:
#Drop other not necessary columns and some which have already been analyzed
data.drop('animals', axis=1,inplace=True)
data.drop('number_cow', axis=1,inplace=True)
data.drop('number_porc', axis=1,inplace=True)
data.drop('number_chicken', axis=1,inplace=True)
data.drop('number_small_ruminants', axis=1,inplace=True)

In [52]:
#This function calculates the average mean of a portion of a total
def mean_average(column,column_tot,df):
    l_mean=[]
    
    for index, row in df.iterrows():
        
        if row[column_tot]!=0:
            l_mean.append(row[column]/row[column_tot]*100)
        else:
            l_mean.append(0)
            
    av_mean=sum(l_mean)/len(df)
            
    return av_mean
        

In [53]:
#Check average revenue values
#Livestock: oxen_revenue,pork_revenue,small_ruminants_revenue,poultry_and_eggs_revenue,livestock_products_sale_revenue
#Agricultur: cereals_revenue,vegetables_revenue,agricolture_products_sale_revenue
#Other: remittances_revenue,other_revenues_value,small_business_revenue 

data['oxen_revenue']=data['oxen_revenue'].replace(np.nan,0)
data['pork_revenue']=data['pork_revenue'].replace(np.nan,0)
data['small_ruminants_revenue']=data['small_ruminants_revenue'].replace(np.nan,0)
data['poultry_and_eggs_revenue']=data['poultry_and_eggs_revenue'].replace(np.nan,0)
data['livestock_products_sale_revenue']=(data['oxen_revenue']+data['pork_revenue']
                                         +data['small_ruminants_revenue']+data['poultry_and_eggs_revenue'])

#data['livestock_products_sale_revenue'].replace(np.nan,0)

data['cereals_revenue']=data['cereals_revenue'].replace(np.nan,0)
data['vegetables_revenue']=data['vegetables_revenue'].replace(np.nan,0)
data['agricolture_products_sale_revenue']=data['cereals_revenue']+data['vegetables_revenue']

#data['agricolture_products_sale_revenue'].replace(np.nan,0)

data['small_business_revenue']=data['small_business_revenue'].replace(np.nan,0)
data['other_revenues_value']=data['other_revenues_value'].replace(np.nan,0)
data['remittances_revenue']=data['remittances_revenue'].replace(np.nan,0)

data['total_revenue']=(data['livestock_products_sale_revenue']
                  +data['agricolture_products_sale_revenue']
                  +data['small_business_revenue']
                  +data['remittances_revenue']
                  +data['other_revenues_value'])   

# drop very high unrealistic revenue values
for index, row in data.iterrows():
    if row['total_revenue']>2000000: 
        print(index,'high value',row['total_revenue'])
        data.drop(index,inplace=True)

print('Total revenues: average %8.2f - max %8.2f - min %8.2f' 
      %(data['total_revenue'].mean(),data['total_revenue'].max(),data['total_revenue'].min()) )

print('Percentage average cereal revenue: %3.1f%%' %(mean_average('cereals_revenue','total_revenue',data)) )

print('Percentage average vegetables revenue: %3.1f%%' %(mean_average('vegetables_revenue','total_revenue',data)) )

print('Percentage average agricultural products revenue: %3.1f%%' 
      %(mean_average('agricolture_products_sale_revenue','total_revenue',data)) )

print('Percentage average small ruminants revenue: %3.1f%%' 
      %(mean_average('small_ruminants_revenue','total_revenue',data)) )

print('Percentage average poultry and eggs revenue: %3.1f%%' 
      %(mean_average('poultry_and_eggs_revenue','total_revenue',data)) )

print('Percentage average pork revenue: %3.1f%%' 
      %(mean_average('pork_revenue','total_revenue',data)) )

print('Percentage average cow revenue: %3.1f%%' 
      %(mean_average('oxen_revenue','total_revenue',data)) )

print('Percentage average livestock products revenue: %3.1f%%' 
      %(mean_average('livestock_products_sale_revenue','total_revenue',data)) )

print('Percentage average small_business revenue: %3.1f%%' 
      %(mean_average('small_business_revenue','total_revenue',data)) )

print('Percentage average remittances revenue: %3.1f%%' 
      %(mean_average('remittances_revenue','total_revenue',data)) )

print('Percentage average other revenue: %3.1f%%' 
      %(mean_average('other_revenues_value','total_revenue',data)) )

510 high value 3792000.0
891 high value 4850000.0
Total revenues: average 152573.27 - max 1990000.00 - min     0.00
Percentage average cereal revenue: 42.0%
Percentage average vegetables revenue: 4.3%
Percentage average agricultural products revenue: 46.4%
Percentage average small ruminants revenue: 4.1%
Percentage average poultry and eggs revenue: 4.0%
Percentage average pork revenue: 7.9%
Percentage average cow revenue: 0.2%
Percentage average livestock products revenue: 16.3%
Percentage average small_business revenue: 21.4%
Percentage average remittances revenue: 5.2%
Percentage average other revenue: 10.6%


In [54]:
# We keep only the total revenues column
data.drop('oxen_revenue', axis=1, inplace=True)
data.drop('pork_revenue', axis=1, inplace=True)
data.drop('small_ruminants_revenue', axis=1, inplace=True)
data.drop('poultry_and_eggs_revenue', axis=1, inplace=True)
data.drop('livestock_products_sale_revenue', axis=1, inplace=True)
data.drop('cereals_revenue', axis=1, inplace=True)
data.drop('vegetables_revenue', axis=1, inplace=True)
data.drop('agricolture_products_sale_revenue', axis=1, inplace=True)
data.drop('small_business_revenue', axis=1, inplace=True)
data.drop('other_revenues_value', axis=1, inplace=True)
data.drop('remittances_revenue', axis=1, inplace=True)
data.drop('other_revenues', axis=1, inplace=True)
data.drop('total_revenues', axis=1, inplace=True) #old column, a new one has been created

In [55]:
#Check average expenses: food,school,your village,construction,agropastoral,medical care,community life,other
data['total_expenditures']=data['total_expenditures'].replace(np.nan,0)
data['food_expenditures']=data['food_expenditures'].replace(np.nan,0)
data['school_fees_expenditures']=data['school_fees_expenditures'].replace(np.nan,0)
data['contruction_expenditures']=data['contruction_expenditures'].replace(np.nan,0)
data['community_life_expenditures']=data['community_life_expenditures'].replace(np.nan,0)
data['agropastoral_expenditures']=data['agropastoral_expenditures'].replace(np.nan,0)
data['medical_care_expenditures']=data['medical_care_expenditures'].replace(np.nan,0)
data['other_expenditures_value']=data['other_expenditures_value'].replace(np.nan,0)


#print(data['total_expenditures'].sum())
#print(data['food_expenditures'].sum()+data['school_fees_expenditures'].sum()
#      +data['contruction_expenditures'].sum()+data['community_life_expenditures'].sum()
#      +data['agropastoral_expenditures'].sum()+data['medical_care_expenditures'].sum()
#      +data['other_expenditures_value'].sum())

data['total_expenses']=(data['food_expenditures']+data['school_fees_expenditures']
                        +data['contruction_expenditures']+data['community_life_expenditures']
                        +data['agropastoral_expenditures']+data['medical_care_expenditures']
                        +data['other_expenditures_value'])

# drop very high unrealistic expenditure values
for index, row in data.iterrows():
    if row['total_expenses']>1000000: 
        print(index,'high value',row['total_expenses'])
        data.drop(index,inplace=True)


print('Average annual expenses: %8.2f' %(data['total_expenses'].mean()))
print('Percentage average food expenses: %3.1f%%' 
      %(mean_average('food_expenditures','total_expenses',data)) )
print('Percentage average school fees expenses: %3.1f%%' 
      %(mean_average('school_fees_expenditures','total_expenses',data)) )
print('Percentage average contruction expenses: %3.1f%%' 
      %(mean_average('contruction_expenditures','total_expenses',data)) )
print('Percentage average community life expenses: %3.1f%%' 
      %(mean_average('community_life_expenditures','total_expenses',data)) )
print('Percentage average agropastoral expenses: %3.1f%%' 
      %(mean_average('agropastoral_expenditures','total_expenses',data)) )
print('Percentage average medical care expenses: %3.1f%%' 
      %(mean_average('medical_care_expenditures','total_expenses',data)) )
print('Percentage average other expenses: %3.1f%%' 
      %(mean_average('other_expenditures_value','total_expenses',data)) )

885 high value 1855200.0
1179 high value 3750400.0
1218 high value 1025750.0
1869 high value 1012100.0
Average annual expenses: 99287.83
Percentage average food expenses: 38.0%
Percentage average school fees expenses: 14.3%
Percentage average contruction expenses: 3.0%
Percentage average community life expenses: 7.9%
Percentage average agropastoral expenses: 13.0%
Percentage average medical care expenses: 11.3%
Percentage average other expenses: 12.5%


In [56]:
# We keep only the total expenses column
data.drop('food_expenditures', axis=1, inplace=True)
data.drop('school_fees_expenditures', axis=1, inplace=True)
data.drop('contruction_expenditures', axis=1, inplace=True)
data.drop('community_life_expenditures', axis=1, inplace=True)
data.drop('agropastoral_expenditures', axis=1, inplace=True)
data.drop('medical_care_expenditures', axis=1, inplace=True)
data.drop('other_expenditures_value', axis=1, inplace=True)
data.drop('other_expenditures', axis=1, inplace=True)
data.drop('total_expenditures', axis=1, inplace=True)#New column 'total_expenses' has been created

## Analyze further info about beneficiaries: meals, house properties, means of transport, agricultural equipment

In [57]:
#Check mean number of meals per day and average value:
data['meals_per_day']=data['meals_per_day'].replace(np.nan,0)
data['meals_per_day_value']=data['meals_per_day_value'].replace(np.nan,0)

#Check if there are strange values for meals per day and adjust them to the mean
#x=data[['Unnamed: 0','meals_per_day']].groupby('meals_per_day').count()
#x.sort_values('Unnamed: 0')

#correct unrealistic values setting to mean value
for index in data.index:
    if data.at[index, 'meals_per_day'] > 6:
        data.at[index, 'meals_per_day'] = 3

print('Average number of meals per day: %1i' %data['meals_per_day'].mean())
        
#Check if there are strange values for meals per day and adjust them to the upper reasonable level
#x=data[['Unnamed: 0','meals_per_day_value']].groupby('meals_per_day_value').count()
#x.sort_values('meals_per_day_value')

#correct unrealistic values
for index in data.index:
    if data.at[index, 'meals_per_day_value'] > 100000:
        data.at[index, 'meals_per_day_value'] = 50000

print('Average value of meals per day: %5.1f' %data['meals_per_day_value'].mean())

#Check property house number average:
data['property_houses']=data['property_houses'].replace(np.nan,0)
#x=data[['Unnamed: 0','property_houses']].groupby('property_houses').count()
#x.sort_values('property_houses')

#Check if there are strange values for property house numbers and adjust to the most probable value
for index in data.index:
    if data.at[index, 'property_houses'] > 20:
        data.at[index, 'property_houses'] = 1
        
print('Average number of property houses: %2i' %data['property_houses'].mean())

#Check average property house value:
data['property_houses_value']=data['property_houses_value'].replace(np.nan,0)
print('Average value of property type: %5.1f' %data['property_houses_value'].mean())

#Check motorbike and bike average value:
data['motorbike']=data['motorbike'].replace(np.nan,0)
data['motorbike_value']=data['motorbike_value'].replace(np.nan,0).replace(-1,0)
data['velo']=data['velo'].replace(np.nan,0)
data['velo_value']=data['velo_value'].replace(np.nan,0)

#x=data[['Unnamed: 0','velo']].groupby('velo').count()
#x.sort_values('velo')
#x=data[['Unnamed: 0','velo_value']].groupby('velo_value').count()
#x.sort_values('velo_value')

#Set strange numbers to mean bike number
for index in data.index:
    if data.at[index, 'velo'] > 20:
        data.at[index, 'velo'] = 1
        
#Set strange bike values to upper value       
for index in data.index:
    if data.at[index, 'velo_value'] > 75000:
        data.at[index, 'velo_value'] = 75000

#Calculate mean motorbike value from those who have a motorbike      
i=0
sum_moto=0
for index, row in data.iterrows():
    if row['motorbike'] != 0.0: 
        i+=1
        sum_moto = sum_moto + row['motorbike_value']

moto_percent = i/len(data)*100
moto_av_value=sum_moto/i

#Calculate mean bike value from those who have a bike      
i=0
sum_bike=0
for index, row in data.iterrows():
    if row['velo'] != 0.0: 
        i+=1
        sum_bike = sum_bike + row['velo_value']
        
bike_percent = i/len(data)*100
bike_av_value=sum_bike/i

print('Percent of beneficiaries with motorbikes: %2i%%' %moto_percent)
print('Average motorbike value: %3.1f' %moto_av_value)
print('Percent of beneficiaries with bikes: %2i%%' %bike_percent)
print('Average bike value: %3.1f' %bike_av_value)

#Check agricultural_equipment value
data['agricultural_equipment']=data['agricultural_equipment'].replace(np.nan,'n')
data['agricultural_equipment_value']=data['agricultural_equipment_value'].replace(np.nan,0)
#x=data[['Unnamed: 0','agricultural_equipment']].groupby('agricultural_equipment').count()
#x.sort_values('Unnamed: 0')
for index in data.index: 
    data.at[index,'agricultural_equipment']=1 if data.at[index,'agricultural_equipment'] == 'y' else 0

i=0
sum_agri=0
for index, row in data.iterrows():
    if row['agricultural_equipment'] == 1:
        i+=1
        sum_agri = row['agricultural_equipment_value'] +sum_agri
agri_av_value=sum_agri/i

#x=data[['Unnamed: 0','agricultural_equipment_value']].groupby('agricultural_equipment_value').count()
#x.sort_values('agricultural_equipment_value')
    
print('Percentage of beneficiaries with agricultural equipment: %3.1f%%'
      %(data['agricultural_equipment'].sum()/len(data)*100))
print('Average value of agricultural equipment: %3.1f' %agri_av_value)


Average number of meals per day: 2
Average value of meals per day: 882.0
Average number of property houses:  1
Average value of property type: 79715.3
Percent of beneficiaries with motorbikes:  6%
Average motorbike value: 50495.8
Percent of beneficiaries with bikes: 77%
Average bike value: 12554.8
Percentage of beneficiaries with agricultural equipment: 12.6%
Average value of agricultural equipment: 31967.9


### Check if beneficiaries can assure child's rights

In [58]:
#Check possibility of children feeding, children caring, daughter caring
data['children_feeding']=data['children_feeding'].replace(np.nan,'n')
data['children_caring']=data['children_caring'].replace(np.nan,'n')
data['daughter_caring']=data['daughter_caring'].replace(np.nan,'n')

for index in data.index: 
    data.at[index,'children_feeding']=1 if data.at[index,'children_feeding'] == 'y' else 0
for index in data.index: 
    data.at[index,'children_caring']=1 if data.at[index,'children_caring'] == 'y' else 0
for index in data.index: 
    data.at[index,'daughter_caring']=1 if data.at[index,'daughter_caring'] == 'y' else 0

#x=data[['Unnamed: 0','children_feeding']].groupby('children_feeding').count()
#x.sort_values('children_feeding')
print('Beneficiaries who have difficulties feeding kids:', len(data)-data['children_feeding'].sum())
print('Beneficiaries who have difficulties caring for kids:', len(data)-data['children_caring'].sum())
print('Beneficiaries who take care equally of girls:', data['daughter_caring'].sum())

Beneficiaries who have difficulties feeding kids: 268
Beneficiaries who have difficulties caring for kids: 125
Beneficiaries who take care equally of girls: 1629


In [59]:
#Let's translate civil status of beneficiaries into numbers: (single,married,divorced,widow)--> (0,1,2,3)
data['status']=data['status'].replace(np.nan,'married')
print(card_dict['status'],data.status.unique())
x=data[['Unnamed: 0','status']].groupby('status').count()
x.sort_values('status')

for index in data.index: 
    if data.at[index,'status'] == 'single' :
        data.at[index,'status'] = 0
    elif data.at[index,'status'] == 'married' :
        data.at[index,'status'] = 1
    elif data.at[index,'status'] == 'divorced' :
        data.at[index,'status'] = 2
    elif data.at[index,'status'] == 'widow' :
        data.at[index,'status'] = 3
x=data[['Unnamed: 0','status']].groupby('status').count()
x.sort_values('status')  

4 ['married' 'widow' 'single' 'divorced']


,Unnamed: 0
status,
0,2
1,1243
2,1
3,419


In [60]:
#check job type of beneficiaries
print(card_dict['job_type'],data.job_type.unique())
x=data[['Unnamed: 0','job_type']].groupby('job_type').count()
x.sort_values('job_type')

2 ['informal' 'formal' nan]


,Unnamed: 0
job_type,
formal,6
informal,1629


In [61]:
#checking if house have a water well
data['water_well']=data['water_well'].replace(np.nan,'n')
print(card_dict['water_well'],data.water_well.unique())

for index in data.index:
    data.at[index,'water_well']=1 if data.at[index,'water_well'] == 'y' else 0
    
x=data[['Unnamed: 0','water_well']].groupby('water_well').count()
x.sort_values('water_well')

2 ['n' 'y']


,Unnamed: 0
water_well,
0,470
1,1195


In [62]:
#x=data[['Unnamed: 0','available_land']].groupby('available_land').count()
#x.sort_values('available_land')
#x=data[['Unnamed: 0','facilitator_perception']].groupby('facilitator_perception').count()
#x.sort_values('facilitator_perception')
#x=data[['Unnamed: 0','local_product_quantity']].groupby('local_product_quantity').count()
#x.sort_values('local_product_quantity')
data['your_village_expenses']=data['your_village_expenses'].replace(np.nan,0)
data['available_land']=data['available_land'].replace(np.nan,0)
data['facilitator_perception']=data['facilitator_perception'].replace(np.nan,0)
data['local_product_quantity']=data['local_product_quantity'].replace(np.nan,0)

In [63]:
#drop unnecessary columns
data.drop('household',axis=1,inplace=True)
data.drop('start_livestock_animals',axis=1,inplace=True)
data.drop('origin_village',axis=1,inplace=True)
data.drop('group_member_benefits',axis=1,inplace=True)
data.drop('$value',axis=1,inplace=True)
data.drop('job_type',axis=1,inplace=True)
data.drop('sex',axis=1,inplace=True)
data.drop('children_caring_how',axis=1,inplace=True)
data.drop('beneficiary_wish',axis=1,inplace=True)
data.drop('type_of_document', axis=1,inplace=True)
data.drop('other_main_activity', axis=1,inplace=True)
data.drop('house_nature', axis=1,inplace=True)
data.drop('wish_motivation', axis=1,inplace=True)

In [64]:
data.drop('Unnamed: 0', axis=1,inplace=True)
data.drop('main_activity', axis=1,inplace=True)

In [65]:
print(data.shape)
#data.head()
#data.isnull().any()

(1665, 50)


In [66]:
cols = list(data.columns.values)
cols

['velo',
 'status',
 'latrines',
 'motorbike',
 'velo_value',
 'water_well',
 'group_member',
 'school_level',
 'meals_per_day',
 'available_land',
 'alphabetization',
 'children_caring',
 'daughter_caring',
 'motorbike_value',
 'property_houses',
 'children_feeding',
 'meals_per_day_value',
 'property_houses_value',
 'your_village_expenses',
 'agricultural_equipment',
 'facilitator_perception',
 'local_product_quantity',
 'agricultural_equipment_value',
 'school_dropout',
 'age',
 'num_languages',
 'num_kids',
 'num_kids_registered',
 'num_kids_certified',
 'num_kids_noschool',
 'num_kids_newborn',
 'num_kids_primary',
 'num_kids_secondary',
 'num_kids_left_school',
 'num_girls',
 'num_girls_registered',
 'num_girls_certified',
 'num_girls_noschool',
 'num_girls_newborn',
 'num_girls_primary',
 'num_girls_secondary',
 'num_girls_left_school',
 'num_boys',
 'num_kids_na',
 'num_adults',
 'num_adults_left_school',
 'out_of_school',
 'animals_tot',
 'total_revenue',
 'total_expenses']

In [67]:
data=(data[['age','status','school_level','alphabetization','num_languages','group_member'
            ,'property_houses','property_houses_value','latrines','water_well','available_land'
            ,'velo','velo_value','motorbike','motorbike_value'
            ,'agricultural_equipment','agricultural_equipment_value'
            ,'your_village_expenses','facilitator_perception','local_product_quantity'
            ,'animals_tot','total_revenue','total_expenses'
            ,'meals_per_day','meals_per_day_value','children_caring','daughter_caring','children_feeding'
            ,'num_kids','num_kids_registered','num_kids_certified','num_kids_noschool'
            ,'num_kids_newborn','num_kids_primary','num_kids_secondary'
            ,'num_kids_left_school','num_girls','num_girls_registered','num_girls_certified'
            ,'num_girls_noschool','num_girls_newborn','num_girls_primary'
            ,'num_girls_secondary','num_girls_left_school','num_boys','num_kids_na'
            ,'num_adults','num_adults_left_school','out_of_school']])

In [68]:
#data.drop('num_kids_noschool',axis=1, inplace=True)
#data.drop('num_kids_left_school',axis=1, inplace=True)
#data.drop('num_girls_noschool',axis=1, inplace=True)
#data.drop('num_girls_left_school',axis=1, inplace=True)

In [69]:
data.to_csv('data_clean.csv')